In [11]:
import numpy as np
import astropy
import matplotlib.pyplot as plt
import os
import astropy.io.fits as fits
from astropy.table import Table
from desitarget import desi_mask, bgs_mask, mws_mask, contam_mask
from desiutil.plots import prepare_data, init_sky, plot_sky_circles, plot_sky_binned
from desiutil.log import get_logger
import warnings
import mpl_toolkits
%matplotlib inline

In [12]:
topdir = os.path.join("/global/cscratch1/sd/forero/test_20170829/targets/N20/")
truthfile = os.path.join(topdir, 'truth.fits')
targetfile = os.path.join(topdir, 'targets.fits')
skyfile = os.path.join(topdir, 'sky.fits')
stdstarsbrightfile = os.path.join(topdir, 'standards-bright.fits')
stdstarsdarkfile = os.path.join(topdir, 'standards-dark.fits')

In [13]:
targets  = fits.getdata(targetfile)
truth = fits.getdata(truthfile)
sky = fits.getdata(skyfile)
stdstars_bright  = fits.getdata(stdstarsbrightfile)
stdstars_dark  = fits.getdata(stdstarsdarkfile)

In [ ]:
def qadensity(cat, objtype, targdens=None, max_bin_area=10.0):
    label = '{} (targets/deg$^2$)'.format(objtype)
    if targdens:
        fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    else:
        fig, ax = plt.subplots(1)
    ax = np.atleast_1d(ax)
    
    basemap = init_sky(galactic_plane_color='k', ax=ax[0]);
    basemap, data = plot_sky_binned(cat['RA'], cat['DEC'], max_bin_area=max_bin_area,
                                    clip_lo='!1', cmap='jet', plot_type='healpix', 
                                    label=label, basemap=basemap)
    if targdens:
        dens = data[np.nonzero(data)]
        ax[1].hist(dens, bins=40, histtype='stepfilled', alpha=0.6, label='Observed {} Density'.format(objtype))
        if objtype in targdens.keys():
            ax[1].axvline(x=targdens[objtype], ls='--', color='k', label='Goal {} Density'.format(objtype))
        ax[1].set_xlabel(label)
        ax[1].set_ylabel('Number of Healpixels')
        ax[1].legend(loc='upper left', frameon=False)
        fig.subplots_adjust(wspace=0.2)
    return dens

In [ ]:
# Desired target densities, including contaminants.
targdens = {'ELG': 2400, 'LRG': 350, 'QSO': 260, 'SKY': 1400}

In [ ]:
for obj in ['ELG','LRG','QSO']:
    these = (targets['DESI_TARGET'] & desi_mask.mask(obj)) != 0
    dd = targets[these]
    a = qadensity(dd, obj, targdens=targdens)

In [ ]:
for obj in ['BGS_ANY', 'MWS_ANY']:
    these = (targets['DESI_TARGET'] & desi_mask.mask(obj)) != 0
    dd = targets[these]
    a = qadensity(dd, obj, targdens=targdens)

In [ ]:
a = qadensity(sky, 'SKY', targdens=targdens)

In [ ]:
a = qadensity(stdstars_bright, 'STD_STARS_BRIGHT', targdens=targdens)

In [ ]:
a = qadensity(stdstars_dark, 'STD_STARS_DARK', targdens=targdens)